In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
ledgers = pd.read_csv('ledgers.csv')
ledgers

In [ ]:
transactions = ledgers[ledgers['type'].isin(['receive', 'spend'])]
transactions.loc[transactions['type'] == 'spend', 'amount'] *= -1
transactions = transactions[['amount', 'asset', 'refid']]
transactions = transactions.pivot(columns='asset', index='refid', values='amount')
transactions['purchase_price'] = transactions['ZUSD'] / transactions['XXBT']
transactions.round({'purchase_price': 2})

In [ ]:
SELL_PERCENTAGES = range(0, 6)
sell_prices = []
profits = []

for transaction in transactions.itertuples():
    purchase_price = transaction.purchase_price
    quantity = transaction.XXBT
    percentage_sell_prices = []
    percentage_profits = []
    
    for percentage in SELL_PERCENTAGES:
        # Calculate target sell price
        sell_price = purchase_price * (1 + percentage / 100)
        percentage_sell_prices.append(sell_price)
        
        # Calculate profit for color mapping
        profit = (sell_price - purchase_price) * quantity
        percentage_profits.append(profit)
    
    sell_prices.append(percentage_sell_prices)
    profits.append(percentage_profits)

sell_prices_df = pd.DataFrame(sell_prices, 
                                columns=[f'{p:+d}%' for p in SELL_PERCENTAGES],
                                index=transactions.index)
profits_df = pd.DataFrame(profits, 
                            columns=[f'{p:+d}%' for p in SELL_PERCENTAGES],
                            index=transactions.index)

fig, ax = plt.subplots(figsize=(16, 10))

# Create annotations showing sell prices
annot_data = sell_prices_df.copy()
for col in annot_data.columns:
    annot_data[col] = annot_data[col].apply(lambda x: f'${x:,.0f}')

# Use profit for color but display sell price
sns.heatmap(profits_df, annot=annot_data, fmt='', cmap='RdYlGn', center=0,
            cbar_kws={'label': 'Profit/Loss ($)'}, linewidths=2, linecolor='black',
            ax=ax, vmin=-profits_df.abs().max().max(), 
            vmax=profits_df.abs().max().max(),
            annot_kws={'size': 12, 'weight': 'bold'})

# Customize title
ax.set_title('Bitcoin Sell Price Target Heatmap\n' + 
             'Showing target sell prices at different percentage gains/losses',
             fontsize=16, fontweight='bold', pad=20)

ax.set_ylabel('Purchase', fontsize=13, fontweight='bold')
ax.set_xlabel('Sell Point Percentage', fontsize=13, fontweight='bold')

# Enhanced y-axis labels with purchase details
y_labels = []
for name, price, qty, inv in zip(transactions.index, transactions['purchase_price'],
                                    transactions['XXBT'], transactions['ZUSD']):
    y_labels.append(f"{name}\nBuy: ${price:,.0f}\n{qty} BTC (${inv:,.0f})")

ax.set_yticklabels(y_labels, rotation=0, fontsize=10)
ax.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=11, weight='bold')

plt.tight_layout()
plt.show()